### Market Level Forecasting

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import preprocessing
from preprocessing import preprocessingMLS, featureEngineering
from getGoogleTrends import getGoogleTrendsData
import models
from imp import reload
import matplotlib.ticker as ticker

### Step 1: Read and preprocess data
House market data, Google Trends, and Economic indicators

In [3]:
data_dir = 'index-team-data/denver/'

In [4]:
#read in market data
process1 = preprocessingMLS(data_dir+'listing_dates_with_ct.csv')
monthData = process1.get_monthly_data(columns=['count_sale','count_list','sale_price'],start_date='2016-03', end_date='2020-09')


#get google trends data from API/read in data
#----------------------------------------------------
#gt = getGoogleTrendsData(geo='US-GA',city='Atlanta')
#trend = gt.getData(times=10,date='2016-03-01 2020-09-30',scaled=True)
#trend.to_csv('google_trends_atlanta_scaled.csv')
#----------------------------------------------------
trend = pd.read_csv(data_dir+'google_trends/google_trends_scaled.csv')


#read in economics indicator features, if needed
#econ = pd.read_csv('econ_data.csv')

merged = pd.merge(monthData, trend).drop(['Unnamed: 0'],axis=1)

In [5]:
#create lag features, pct change features and one hot month features with featureEngineering
feature_column_names = merged.columns.delete(0)
fe = preprocessing.featureEngineering(merged,feature_column_names)
fe.create_lag_features(lag_num_list=[1,3])
fe.create_pct_change_features(lag_num_list=[1])
fe.create_month_one_hot()
df = fe.get_dataFrame()
feature_column_names = fe.get_feature_names()
print(len(feature_column_names))

136


### Step 2: Make predictions

### 2.1 predict 3 months ahead -- google trends

In [6]:
# predict 3 months ahead -- google trends
target_col = 'count_sale'
horizon = 3
model = models.rollingModel(df,horizon,target_col,feature_column_names,
                            modelName='linear',lead_target=True)
pred, pred_boot, pred_scale, pred_samples = model.fit_predict(df[feature_column_names], df[target_col])

prediction_horizon 1


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22550.308295654155, tolerance: 4429.343462745098
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3729.035679868073, tolerance: 3538.99419
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4043.501137708925, tolerance: 3920.42119
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number o

prediction_horizon 2


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 41633.13298217836, tolerance: 8617.874098
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26930.66937131038, tolerance: 10536.895928
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 59605.17301178814, tolerance: 6627.588888
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of it

prediction_horizon 3


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23301.001640125607, tolerance: 10454.083412244898
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 30531.41593655072, tolerance: 21436.91093877551
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 50110.437900059456, tolerance: 13249.59067755102
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to incre

### 2.2 predict 3 months ahead -- time series

In [7]:
#predict 3 months ahead -- Time series
model = models.BTSM()
model.fit(df['count_sale'].values.astype(float),0)
pred_ts, pred_ts_scale, pred_ts_samples = model.predict(np.ones(3))

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.
Instructions for updating:
Do not call `graph_parents`.
Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Previously, `initial_step` would return the value passed to the constructor, even if that value was not `Tensor`-like. This behavior is now deprecated. In the future, it will always return a `Tensor`-like object.
Instructions for updating:
The signature for `trace_fn`s passed to `minimize` has changed. Trace functions now take a single `traceable_quantities` argument, which is a `tfp.math.MinimizeTraceableQuantities` namedtuple containing `traceable_quantities.loss`, `traceable_quantities.gradients`, etc. Please update your `trace_

### 2.3 predict 3 months ahead -- linear combination

In [8]:
#combine the two prediciton
#params may differ in different market
params = [0.6,0.4]
pred_com = params[0]*np.array(pred)+params[1]*pred_ts
pred_com_scale = params[0]*pred_scale+params[1]*pred_ts_scale

print(pred_com)

[7971.28543761 6924.11064937 6859.90819297]


### 2.4 predict 6 months ahead  -- number of sales

In [9]:
#predict 6 months ahead
model = models.rollingCombinedModel(df=df,predict_horizon_total=6, predict_horizon=3,
                                    feature_column_names = feature_column_names,
                                   target_column = target_col, modelName='linear',
                                   lead_target=True)

In [10]:
pred6_sale, pred6_scale_sale, pred6_samples_sale = model.make_prediction(df[feature_column_names], df[target_col])

prediction_horizon 1


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4389.9151906662155, tolerance: 2238.5481609756093
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3285.174859775172, tolerance: 3281.0226774999996
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3594.0643464768364, tolerance: 3281.0226774999996
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to inc

prediction_horizon 2


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 44521.90546792423, tolerance: 7396.906642000002
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15568.3840862955, tolerance: 9816.324088
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 70368.37667923032, tolerance: 11593.233408
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number 

prediction_horizon 3


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 39315.710434666864, tolerance: 15362.83278367347
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 67250.33415692172, tolerance: 16155.605281632654
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13560.138692542445, tolerance: 12548.153974358975
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to incr

In [11]:
pred6_samples_sale.shape

(50, 6)

### 2.5 predict 6 months ahead  -- number of new listings

In [12]:
target_col = 'count_list'
#predict 6 months ahead
model = models.rollingCombinedModel(df=df,predict_horizon_total=6, predict_horizon=3,
                                    feature_column_names = feature_column_names,
                                   target_column = target_col, modelName='linear',
                                   lead_target=True)

In [13]:
pred6_listing, pred6_scale_listing, pred6_samples_listing = model.make_prediction(df[feature_column_names], df[target_col])

prediction_horizon 1


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 223433.21180786405, tolerance: 82108.53887058824
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 62138.95745683196, tolerance: 14629.323392156866
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 105590.28419403732, tolerance: 74051.05392941176
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to incre

prediction_horizon 2


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 155604.1610793426, tolerance: 85774.262528
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 105859.38223008439, tolerance: 81853.78061799999
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 115832.11858376255, tolerance: 111971.40128800002
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase t

prediction_horizon 3


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 416461.8348760995, tolerance: 136381.44460000002
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 415846.6691584666, tolerance: 77088.12519183675
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 348159.39816157677, tolerance: 88155.38609795918
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increa

In [14]:
pred6_samples_listing.shape

(50, 6)

### Step 3: Save the prediction samples for further use

In [15]:
for i in range(6):
    save = pd.DataFrame(columns=['number of sales', 'number of listings'])
    save['number of sales'] = pred6_samples_sale[:,i]
    save['number of listings'] = pred6_samples_listing[:,i]
    save.to_csv(data_dir+'model_forecast/'+str(i+1)+'_month_overall_forecast_samples.csv')